In [26]:
import pandas as pd
import numpy as np
import random as r

In [27]:
data = pd.read_csv('/kaggle/input/steel-industry-data/Occupancy_Estimation.csv')

In [28]:
data.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10129 entries, 0 to 10128
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  10129 non-null  object 
 1   Time                  10129 non-null  object 
 2   S1_Temp               10129 non-null  float64
 3   S2_Temp               10129 non-null  float64
 4   S3_Temp               10129 non-null  float64
 5   S4_Temp               10129 non-null  float64
 6   S1_Light              10129 non-null  int64  
 7   S2_Light              10129 non-null  int64  
 8   S3_Light              10129 non-null  int64  
 9   S4_Light              10129 non-null  int64  
 10  S1_Sound              10129 non-null  float64
 11  S2_Sound              10129 non-null  float64
 12  S3_Sound              10129 non-null  float64
 13  S4_Sound              10129 non-null  float64
 14  S5_CO2                10129 non-null  int64  
 15  S5_CO2_Slope       

In [30]:
data.drop(['Date', 'Time'], axis=1, inplace=True)

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
import warnings
warnings.filterwarnings('ignore')

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = data.drop(columns=['Room_Occupancy_Count'])
Y = data['Room_Occupancy_Count']

Scalar = StandardScaler()
Scalar.fit_transform(X)

samples = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=i)
    samples.append((X_train, X_test, y_train, y_test))

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import random

def fitnessFunction(kernel, C, gamma):
    svm = SVC(kernel=kernel, C=C, gamma=gamma,max_iter = 100)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    return accuracy_score(y_pred, y_test)

kernel = ['linear', 'poly', 'rbf', 'sigmoid']
bestAccuracy = []
bestKernel = []
bestC = []
bestGamma = []
iteration = 100

for i in range(len(samples)):
    best_accuracy= 0
    best_C = 0
    best_gamma = 0
    best_kernel = None
    for k in kernel:
        X_train_random, X_test_random, y_train_random, y_test_random = samples[i]
        C = np.random.uniform(0, 10)
        gamma = np.random.uniform(0, 10)
        score = fitnessFunction(k, C, gamma)
        if score>best_accuracy:
            best_accuracy = round(score,2)
            best_C = round(C,2)
            best_gamma = round(gamma, 2)
            best_kernel  = k
    bestAccuracy.append(best_accuracy)
    bestC.append(best_C)
    bestGamma.append(best_gamma)
    bestKernel.append(best_kernel)
    print('Best Accuracy = ', best_accuracy, 'Best Kernel = ', best_kernel, 'Best C = ', C, 'Best Gamma = ', gamma)

Best Accuracy =  0.95 Best Kernel =  poly Best C =  2.2692504240953824 Best Gamma =  7.0937611966813865
Best Accuracy =  0.96 Best Kernel =  rbf Best C =  4.11827556765656 Best Gamma =  6.0826586150854585
Best Accuracy =  0.96 Best Kernel =  rbf Best C =  4.4206284782087115 Best Gamma =  6.754754938975202
Best Accuracy =  0.92 Best Kernel =  linear Best C =  2.627976819213642 Best Gamma =  2.6131081828162572
Best Accuracy =  0.92 Best Kernel =  linear Best C =  4.890080398386063 Best Gamma =  0.9922188484587402
Best Accuracy =  0.94 Best Kernel =  poly Best C =  9.855869060429844 Best Gamma =  5.083882643796289
Best Accuracy =  0.93 Best Kernel =  poly Best C =  1.6402893402369345 Best Gamma =  8.374941024649157
Best Accuracy =  0.92 Best Kernel =  linear Best C =  2.7420705229156974 Best Gamma =  0.3666314960608885
Best Accuracy =  0.92 Best Kernel =  linear Best C =  3.594944871700557 Best Gamma =  1.3739648962653417
Best Accuracy =  0.94 Best Kernel =  poly Best C =  2.6338404144021

In [34]:
data = {
    'Sample': ['Sample'+str(i+1) for i in range(len(samples))],
    'BestAccuracy': bestAccuracy,
    'BestC': bestC,
    'BestGamma': bestGamma,
    'BestKernel': bestKernel
}
df = pd.DataFrame(data)
print(df)

     Sample  BestAccuracy  BestC  BestGamma BestKernel
0   Sample1          0.95   2.23       7.59       poly
1   Sample2          0.96   0.34       4.77        rbf
2   Sample3          0.96   0.08       5.29        rbf
3   Sample4          0.92   5.45       0.87     linear
4   Sample5          0.92   3.71       9.63     linear
5   Sample6          0.94   9.04       0.10       poly
6   Sample7          0.93   7.73       5.35       poly
7   Sample8          0.92   4.65       1.41     linear
8   Sample9          0.92   6.17       2.65     linear
9  Sample10          0.94   5.52       9.30       poly
